In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model, models
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Reshape, Conv2DTranspose, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import numpy as np
import seaborn as sns
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pdb
import random
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import SGD,RMSprop
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


In [ ]:
# path = r'D:\MILCOM_2025\Spec_SNR-5'
# path = r'D:\MILCOM_2025\Spec_SNR0'
# path = r'D:\MILCOM_2025\Spec_SNR5'
# path = r'D:\MILCOM_2025\Spec_BIN_data_SNR-5'
# path = r'D:\MILCOM_2025\Spec_BIN_data_SNR_0'
# path = r'D:\MILCOM_2025\Spec_BIN_data_SNR5'
# path = r'D:\MILCOM_2025\IQ_BIN_data_SNR-5'
# path = r'D:\MILCOM_2025\IQ_BIN_data_SNR_0'
# path = r'D:\MILCOM_2025\IQ_BIN_data_SNR5'
# path = r'D:\MILCOM_2025\Real_Data_All'
# path = r'D:\MILCOM_2025\Real_BIN_Data'
path = r'D:\MILCOM_2025\Spec_SNR-5_binary'


In [ ]:

save_dir = r"D:\MILCOM_2025\Results"

In [ ]:
import os
from glob import glob

# Get all file paths
all_files = sorted(glob(os.path.join(path, '*', '*')))  # 'path' points to your root folder

# Extract labels (folder names)
all_labels = [os.path.basename(os.path.dirname(f)) for f in all_files]


In [ ]:
img_height = 256
img_width = 256
batch_size = 32

In [ ]:
# Step 1: Get all file paths and labels
all_files = sorted(glob(os.path.join(path, '*', '*')))
all_labels = [os.path.basename(os.path.dirname(f)) for f in all_files]
class_names = sorted(set(all_labels))
label_to_index = {label: idx for idx, label in enumerate(class_names)}
all_indices = [label_to_index[label] for label in all_labels]

# Step 2: Convert to numpy arrays
all_files = np.array(all_files)
all_indices = np.array(all_indices)

# Step 3: Strict split — 50% train, 15% val, 35% test (or any ratio you want)
X_temp, X_test, y_temp, y_test = train_test_split(all_files, all_indices, test_size=0.15, random_state=42, stratify=all_indices)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)  # 0.23 of 65% ≈ 15%

# Step 4: Helper to load and preprocess image
def preprocess_image(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [img_height, img_width])
    image.set_shape((img_height, img_width, 3))
    label = tf.one_hot(label, depth=len(class_names))
    return image, label

# Step 5: Build tf.data.Dataset for each split
def build_dataset(file_paths, labels, is_train=True):
    ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    ds = ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if is_train:
        ds = ds.shuffle(1000)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = build_dataset(X_train, y_train, is_train=True)
val_ds = build_dataset(X_val, y_val, is_train=False)
test_ds = build_dataset(X_test, y_test, is_train=False)



In [ ]:
# Confirm no overlaps across splits
def assert_disjoint(a, b, name1, name2):
    overlap = set(a).intersection(set(b))
    assert len(overlap) == 0, f"Data leakage detected between {name1} and {name2}"

assert_disjoint(X_train, X_val, "train", "val")
assert_disjoint(X_train, X_test, "train", "test")
assert_disjoint(X_val, X_test, "val", "test")
print("✅ All splits are 100% disjoint — no data leakage.")

In [ ]:
class_names = sorted(set(all_labels))
print(class_names)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = len(class_names)

In [ ]:
def create_vit_classifier(
    input_shape=(256, 256, 3),
    patch_size=16,
    projection_dim=64,
    transformer_layers=4,
    num_heads=4,
    mlp_head_units=[128],
    num_classes=num_classes,
    dropout_rate=0.1,
):
    # Number of patches
    num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)

    # Input layer
    inputs = layers.Input(shape=input_shape)

    # Patch creation and projection
    x = layers.Conv2D(filters=projection_dim,
                      kernel_size=patch_size,
                      strides=patch_size,
                      padding='valid')(inputs)
    x = layers.Reshape((num_patches, projection_dim))(x)

    # Positional embedding
    positions = tf.range(start=0, limit=num_patches, delta=1)
    pos_embed = layers.Embedding(input_dim=num_patches, output_dim=projection_dim)(positions)
    x = x + pos_embed

    # Transformer blocks
    for _ in range(transformer_layers):
        # Layer normalization + MSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x1, x1)
        x2 = layers.Add()([x, attention_output])

        # Layer normalization + MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        mlp_output = layers.Dense(projection_dim * 2, activation='gelu')(x3)
        mlp_output = layers.Dropout(dropout_rate)(mlp_output)
        mlp_output = layers.Dense(projection_dim)(mlp_output)
        x = layers.Add()([x2, mlp_output])

    # Final MLP head
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(dropout_rate)(x)

    for units in mlp_head_units:
        x = layers.Dense(units, activation='gelu')(x)
        x = layers.Dropout(dropout_rate)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs=inputs, outputs=outputs)

# Build and compile model
model = create_vit_classifier()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train as before
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=200)

In [ ]:
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

In [ ]:
from tensorflow.keras.models import Model

# Create a model that outputs embeddings from the penultimate layer
embedding_model = Model(inputs=model.input, outputs=model.layers[-2].output)

# Extract images and labels from the test dataset
features = []
labels = []

for batch_images, batch_labels in test_ds:
    batch_features = embedding_model.predict(batch_images)
    features.append(batch_features)
    labels.append(batch_labels.numpy())

# Concatenate all batches
import numpy as np

features = np.concatenate(features, axis=0)
labels = np.concatenate(labels, axis=0)


from sklearn.manifold import TSNE

# Initialize t-SNE with desired parameters
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42)

# Fit and transform the features
tsne_results = tsne.fit_transform(features)



import matplotlib.pyplot as plt

# Convert one-hot encoded labels to class indices if necessary
if labels.ndim > 1:
    labels = np.argmax(labels, axis=1)

# Create a scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=labels, cmap='tab10', alpha=0.7)
plt.legend(*scatter.legend_elements(), title="Classes")
plt.title("t-SNE Visualization of ViT Embeddings")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.grid(True)
plt.show()


In [ ]:
model.summary()

In [ ]:
# Plot Training and Validation Accuracy
plt.figure(figsize=(8, 6))

# Plot the accuracy curves
plt.plot(training_accuracy, label='Training Accuracy', linestyle='--', linewidth=2)  # Thicker dotted curve
plt.plot(validation_accuracy, label='Validation Accuracy', linewidth=2)  # Thicker solid curve

# Adding labels, title, and legend with larger and bold font
plt.xlabel('Epochs', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy', fontsize=14, fontweight='bold')
plt.title('Training and Validation Accuracy', fontsize=16, fontweight='bold')

# Make the legend font size larger and bold using 'prop'
plt.legend(fontsize=12, prop={'weight':'bold'})

# Make tick labels bold and increase font size
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')

# Adding a grid with both major and minor grid lines
plt.grid(True, which='both', linewidth=0.3)  # Smaller squares with thinner grid lines
plt.minorticks_on()  # Enable minor ticks
plt.tight_layout()
plt.savefig(os.path.join(save_dir,"training_val_accuracy_curve_SNR5_BIN_data_ViT.png"), dpi=600)

# Display the plot
plt.show()

In [ ]:
# Assuming that 'history' is the result of model training
training_loss = history.history['loss']
validation_loss = history.history['val_loss']


# Plot Training and Validation Accuracy
plt.figure(figsize=(8, 6))

# Plot the accuracy curves
plt.plot(training_loss, label='Training Loss', linestyle='--', linewidth=2)  # Thicker dotted curve
plt.plot(validation_loss, label='Validation Loss', linewidth=2)  # Thicker solid curve

# Adding labels, title, and legend with larger and bold font
plt.xlabel('Epochs', fontsize=14, fontweight='bold')
plt.ylabel('Loss', fontsize=14, fontweight='bold')
plt.title('Training and Validation Loss', fontsize=16, fontweight='bold')

# Make the legend font size larger and bold using 'prop'
plt.legend(fontsize=12, prop={'weight':'bold'})

# Make tick labels bold and increase font size
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')

# Adding a grid with both major and minor grid lines
plt.grid(True, which='both', linewidth=0.3)  # Smaller squares with thinner grid lines
plt.minorticks_on()  # Enable minor ticks
plt.tight_layout()
plt.savefig(os.path.join(save_dir,"training_val_Loss_curve_SNR5_BIN_data_ViT.png"), dpi=600)
# Display the plot
plt.show()

In [ ]:
# Step 1: Make predictions on the validation dataset
# Get the true labels and the predicted labels
y_true = []
y_pred = []

for images, labels in test_ds:
    y_true.extend(np.argmax(labels.numpy(), axis=1))  # True labels
    predictions = model.predict(images)
    y_pred.extend(np.argmax(predictions, axis=1))  # Predicted labels

# Convert to numpy arrays for confusion matrix
y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 7))

# Increase annotation size and make them bold
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, cbar=False, 
            annot_kws={"size": 20, "fontweight": "bold"})  # Set annotation font size and bold


# Increase font size for axis labels and title
plt.xlabel('Predicted Label ', fontsize=16, fontweight = 'bold')
plt.ylabel('True Label', fontsize=16, fontweight = 'bold')

# Additional step: Use plt.setp() to make tick labels bold
plt.setp(plt.gca().get_xticklabels(), fontweight='bold')  # Set x-tick labels to bold
plt.setp(plt.gca().get_yticklabels(), fontweight='bold')  # Set y-tick labels to bold

# Increase tick label font size
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Save the figure with 400 DPI
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "confusion_matrix_SNR5_BIN_data_ViT.png"), dpi=600)

plt.show()

In [ ]:
misclassified_indices = np.where(y_true != y_pred)[0]
print("Misclassified sample indices:", misclassified_indices)
